In [2]:
import sys
import otter

# try:
#   import otterdd
# except ImportError:
#     %pip install otter-grader
#     import otter

grader = otter.Notebook("HWB1.ipynb")

## 🧠 Bonus: Converting Digital Circuits to Transition Systems

### 🔍 Objective

In this assignment, you will implement a method to convert a simple **digital circuit** into a **transition system**. The goal is to capture the behavior of a circuit under all possible input combinations and internal register states.

---

### 🧩 Problem Setup

You are given a class `Circuit` with the following structure:

```python
class Circuit:
    def __init__(self, X, R, Y, update_registers, compute_outputs):
        ...
```

- `X`: Number of binary input bits
- `R`: Number of binary registers (internal state bits)
- `Y`: Number of binary output bits
- `update_registers`: A function that computes the next register state based on inputs and current registers
- `compute_outputs`: A function that computes outputs based on inputs and current registers

Your task is to implement the method:

```python
def to_transition_system(self) -> TransitionSystem:
    ...
```

This method constructs a transition system representing the circuit’s behavior.

---


### 🔧 Transition System Specification

The transition system should include the following components:

#### ✅ States (`S`)
- A state is a pair `(X, R)`, representing a combination of input values and register values.
- Enumerate all possible Boolean combinations.
- Input values and register values should be tuples of Booleans.

#### ✅ Actions (`Act`)
- Actions correspond to all possible values of the input vector `X`.

#### ✅ Transitions (`T`)
- Each transition `(s, a, s')` represents moving from state `s` to `s'` by applying input `a`.
- `s = (X_old, R_old)`, `a = X_new`, `s' = (X_new, update_registers(X_old, R_old))`

#### ✅ Initial States (`I`)
- All states where the register vector is initialized to all `False` (`0`), and `X` can be any combination.

#### ✅ Atomic Propositions (`AP`)
- APs should include:
  - Each input bit: `x1`, `x2`, ..., `xN`
  - Each register bit: `r1`, `r2`, ..., `rM`
  - Each output bit: `y1`, `y2`, ..., `yK`

#### ✅ Labeling Function (`L`)
- Labels for a state `(X, R)` should include:
  - Inputs `xi` that are `1`
  - Registers `ri` that are `1`
  - Outputs `yi` that are `1`


---


### 🧪 Example

For a circuit with:
- 1 input: `X = 1`
- 1 register: `R = 1`
- 1 output: `Y = 1`

If `update_registers(X, R)` = `X`, and `compute_outputs(X, R)` = `R`, then:

- Initial states: `[(True, False), (False, False)]`
- Action space: `[(True,), (False,)]`
- A transition from `(True, False)` with action `(False,)` goes to `(False, True)`.

---

In [31]:
# Add your imports here
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from systems import TransitionSystem
from itertools import product

class Circuit:
    def __init__(self, X, R, Y, update_registers, compute_outputs):
        self.X = X
        self.R = R
        self.Y = Y
        self._update_registers = update_registers
        self._compute_outputs = compute_outputs

    def update_registers(self, X, R):
        """
        Evaluates the circuit given input values and register values.

        :return: new_registers
        """
        return self._update_registers(X, R)

    def compute_outputs(self, X, R):
        """
        Computes the output values of the circuit given input values and register values.

        :return:  output_values
        """
        return self._compute_outputs(X, R)

    def to_transition_system(self):
        x_combinations = list(product((True, False), repeat=self.X))
        r_combinations = list(product((True, False), repeat=self.R))
        S = {(X, R) for X in x_combinations for R in r_combinations}
        Act = x_combinations
        I = {(X, (False,)*self.R) for X in x_combinations}
        T = {(s_from, a, (a, self.update_registers(s_from[0], s_from[1]))) for s_from in S for a in Act}
        ap = {f'x{i+1}' for i in range(self.X)} | {f'r{i+1}' for i in range(self.R)} | {f'y{i+1}' for i in range(self.Y)}
        def get_labels(s):
            """
            Returns the labels of a state.

            :param s: state
            :return: labels
            """
            return {f'x{i+1}' for i in range(self.X) if s[0][i]} | {f'r{i+1}' for i in range(self.R) if s[1][i]} | {f'y{i+1}' for i in range(self.Y) if self.compute_outputs(s[0], s[1])[i]}
        L = {s: get_labels(s) for s in S}

        return TransitionSystem(states=S, actions=Act, initial_states=I, transitions=T, atomic_props=ap, labeling_map=L)
        


## 🔧 Example: Using `Circuit` and Converting to a `TransitionSystem`

This example demonstrates how to define a simple digital circuit, convert it into a transition system, and inspect its components.

### 🧩 Step 1: Define Update and Output Functions

```python
# Update function: register stores the current input
def update_registers(X, R):
    return (X[0],)

# Output function: output is equal to the register value
def compute_outputs(X, R):
    return (R[0],)
```

---

### 🧩 Step 2: Create the Circuit and Convert

```python
# Create a circuit with 1 input, 1 register, 1 output
circuit = Circuit(X=1, R=1, Y=1, update_registers=update_registers, compute_outputs=compute_outputs)

# Convert the circuit to a transition system
ts = circuit.to_transition_system()
```

---

### 🔍 Step 3: Explore the Transition System

```python
# Print states, actions, initial states, and transitions
print("States:", ts.S)
print("Actions:", ts.Act)
print("Initial States:", ts.I)

print("Some Transitions:")
for t in list(ts.Transitions)[:4]:
    print("  ", t)

# Print labels for one state
print("Labels for first state:", ts.L(next(iter(ts.S))))
```

---

### ✅ Expected Output (Example)

```
States: [((False,), (False,)),
         ((False,), (True,)),
         ((True,), (False,)),
         ((True,), (True,))]
Actions: {(False,), (True,)}
Initial States: {((False,), (False,)),
                 ((True,), (False,))}
Some Transitions:
   (((False,), (False,)), (True,), ((True,), (False,)))
   (((False,), (True,)), (True,), ((True,), (False,)))
   (((True,), (False,)), (False,), ((False,), (True,)))
   (((True,), (True,)), (False,), ((False,), (True,)))
Labels for first state: {'x1', 'r1', 'y1'}
```

This illustrates that:
- Each transition updates the register based on the **previous input**
- Labels correctly reflect inputs (`x1`), registers (`r1`), and outputs (`y1`) that are `True`



In [32]:
# Define the update function: the register stores the current input
def update_registers(X, R):
    return (X[0],)  # one-bit register updated to match input

# Define the output function: output is just the current register value
def compute_outputs(X, R):
    return (R[0],)

# Create the circuit: 1 input, 1 register, 1 output
circuit = Circuit(X=1, R=1, Y=1, update_registers=update_registers, compute_outputs=compute_outputs)

# Convert to transition system
ts = circuit.to_transition_system()

# Print details
print("States:", ts.S)
print("Actions:", ts.Act)
print("Initial States:", ts.I)
print("Some Transitions:")
for t in list(ts.Transitions)[:4]:
    print("  ", t)
print("Labels for first state:", ts.L(next(iter(ts.S))))

States: {((True,), (True,)), ((False,), (False,)), ((True,), (False,)), ((False,), (True,))}
Actions: {(False,), (True,)}
Initial States: {((False,), (False,)), ((True,), (False,))}
Some Transitions:
   (((True,), (False,)), (False,), ((False,), (True,)))
   (((False,), (False,)), (True,), ((True,), (False,)))
   (((True,), (False,)), (True,), ((True,), (True,)))
   (((False,), (False,)), (False,), ((False,), (False,)))
Labels for first state: {'y1', 'r1', 'x1'}


In [33]:
grader.check("q1")

q1 results: All test cases passed!

## 🎮 Question 2: 7-Boom Counter Circuit

### 🔍 Objective

In this task, you will implement a digital circuit that models the classic **"7-Boom"** game logic:
- Count upward from 0 to 7
- When reaching 7, say "Boom!", then reset to 0

You will model this behavior in a `Circuit` object and convert it to a transition system.

---

### 🧩 Circuit Specification

- **Inputs (`X`)**: 1 bit
  - `x = 1` means increment the counter
  - `x = 0` means hold the current value

- **Registers (`R`)**: 3 bits
  - These bits represent a counter from `0` to `7` (in binary)

- **Output (`Y`)**: 1 bit
  - `y = 1` **only when** the counter reaches 8 (i.e., after 7 + 1)
  - Once the counter reaches 8, it immediately resets to 0

---

### 🧪 Behavior Examples

| Step | Input `x` | Register (binary) | Output `y` |
|------|-----------|-------------------|------------|
| 0    | -         | `000` (0)         | 0          |
| 1    | 1         | `001` (1)         | 0          |
| 2    | 1         | `010` (2)         | 0          |
| ...  | ...       | ...               | ...        |
| 6    | 1         | `110` (6)         | 0          |
| 7    | 1         | `111` (7)         | **1** Boom!|
| 8    | 1         | `000` (0)         | 0          |
| 9    | 0         | `000` (0)         | 0          |

---

### ✅ Task 1

Implement the following functions:

```python
def update_registers_7boom(X, R) -> Tuple[bool, bool, bool]:
    ...
```

```python
def compute_outputs_7boom(X, R) -> Tuple[bool]:
    ...
```

```python
def seven_boom_circuit() -> Circuit:
    ...
```

Where:
- You define `update_registers(X, R)` to implement the counting and reset logic
- You define `compute_outputs(X, R)` to output `y = 1` only when counter == 7

Then, test your circuit using `.to_transition_system()` or `.plot()`.

---

In [44]:
def update_registers_7boom(X, R):
    """
    Update the 3-bit register based on the input x (either 0 or 1).

    Parameters:
        X (Tuple[bool]): Input tuple of length 1 (True = increment, False = no-op)
        R (Tuple[bool, bool, bool]): 3-bit register representing an integer from 0 to 7

    Returns:
        Tuple[bool, bool, bool]: The updated 3-bit register (reset to 0 if value becomes 8)
    """
    return (not R[0], R[0] ^ R[1], (R[1] & (not (R[1] ^ R[0]))) ^ R[2]) if all(X) else R



def compute_outputs_7boom(X, R):
    """
    Compute the output of the 7-Boom circuit.

    Parameters:
        X (Tuple[bool]): Input tuple of length 1
        R (Tuple[bool, bool, bool]): 3-bit register

    Returns:
        Tuple[bool]: Output y = 1 iff the register value is 7
    """
    return (True,) if all(R) else (False,)


def seven_boom_circuit():
    """
    Construct and return a Circuit object representing the 7-Boom counter.

    - Inputs: 1-bit (x)
    - Registers: 3-bit counter (to count 0 through 7)
    - Output: 1-bit y = 1 only when counter reaches 8 (then resets)

    Returns:
        Circuit: A fully configured Circuit object for the 7-Boom logic
    """
    return Circuit(
        X=1,  # 1 input
        R=3,  # 3 registers (3 bits)
        Y=1,  # 1 output
        update_registers=update_registers_7boom,
        compute_outputs=compute_outputs_7boom
    )

## 🔍 Task 2

### Objective

Now that you've implemented the 7-Boom circuit, it's time to analyze how many **distinct states** are reachable in its transition system.

---

### 🔧 Task

Implement the function:

```python
def count_reachable_states() -> int:
    ...
```

---


In [46]:
def count_reachable_states() -> int:
    """
    Constructs the transition system of the 7-Boom circuit and returns
    the number of reachable states from the initial states.
    """
    ts = seven_boom_circuit().to_transition_system()
    reachable_states = ts.reach()
    return len(reachable_states)

In [47]:
grader.check("q2")

q2 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)